In [67]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import Huber
import tensorflow as tf

In [68]:
df = pd.read_csv('./input/copper_returns_5d_final.csv')

In [69]:
features = df.drop(['Date', 'LMCADY_std_5d', 'LMCADY_acu_5d'], axis=1)

scaler = StandardScaler()
scaled_features = scaler.fit_transform(df.drop(['Date','LMCADY_std_5d','LMCADY_acu_5d'], axis=1))
features = scaled_features

print(features.shape)


(5539, 8)


In [70]:
def crearSecuencias(data, n_steps):
    X, y = [], []
    print(data)
    try:
        data = data.values  # Asegurarse de que 'data' es un array de NumPy
    except:
        pass
    for i in range(n_steps, len(data)):
        X.append(data[i-n_steps:i, :-2])  # las variables excepto los target
        y.append(data[i, -2:])            # los target
    return np.array(X), np.array(y)

In [71]:
n_steps = 25  # ventana modificable
X, y = crearSecuencias(features, n_steps)
print(X.shape, y.shape)


[[ 8.88174336e-02  7.44158059e-02 -1.61553556e-01 ...  1.40393504e-01
  -2.72839869e-01  4.81635337e-01]
 [-3.41047918e-02 -2.12838525e-04  4.18921718e-01 ...  2.63621263e-01
  -2.48888913e-01 -1.54865105e-02]
 [ 6.01515341e-01  5.77035466e-01 -8.40686743e-01 ...  2.82472734e-01
   7.09955572e-01  1.51088802e-01]
 ...
 [-6.57712049e-01 -6.09357555e-01  1.02160791e+00 ... -1.83950901e+00
   2.55342740e-02 -7.27844599e-02]
 [-8.00281396e-01 -8.02392606e-01  1.03697816e+00 ... -3.61342573e-01
  -1.35527210e+00 -7.46369564e-01]
 [-1.16351169e+00 -1.18211667e+00 -2.23029805e+00 ... -2.30137329e+00
   5.64594036e-01 -3.97293980e-01]]
(5514, 25, 6) (5514, 2)


In [72]:
# verificar que haya secuencia
print(X[0])
print(X[1])

[[ 8.88174336e-02  7.44158059e-02 -1.61553556e-01  2.59411252e-01
   5.90748707e-01  1.40393504e-01]
 [-3.41047918e-02 -2.12838525e-04  4.18921718e-01  2.81169283e-01
  -5.75793243e-01  2.63621263e-01]
 [ 6.01515341e-01  5.77035466e-01 -8.40686743e-01  1.40406932e-01
   7.23844766e-01  2.82472734e-01]
 [-1.45035825e-01 -1.18538239e-01 -1.24529634e+00 -2.52222565e-01
  -1.97613112e-01 -6.61487364e-01]
 [ 3.49823992e-01  3.68778009e-01  8.66286542e-01 -7.27015508e-01
   3.28656972e-01 -5.95866517e-01]
 [ 7.92010537e-01  8.00334113e-01 -4.83857667e-01 -4.54989665e-01
   7.89502991e-02 -2.20061013e-01]
 [-1.04965059e-01 -1.08222140e-01 -3.03676332e-01 -5.11032697e-01
  -4.85731323e-01 -4.85666739e-01]
 [-1.25501934e+00 -1.24386983e+00 -1.42726206e-01  5.74634106e-01
  -1.16455026e-01  6.84052058e-01]
 [ 2.88300845e-01  3.58056712e-01 -1.15218210e+00  2.65487670e-01
  -9.24504706e-02  3.67942343e-01]
 [-1.87205616e+00 -1.86481049e+00  3.43574128e-01  1.84840452e-01
  -8.03081233e-01  7.4307

In [73]:
y

array([[-0.30619194,  0.04646557],
       [-0.1418456 ,  0.88693276],
       [-0.40015258, -1.07698423],
       ...,
       [ 0.02553427, -0.07278446],
       [-1.3552721 , -0.74636956],
       [ 0.56459404, -0.39729398]])

In [74]:
# desviación estándar de los target
np.std(y, axis=0)

array([1.00123389, 1.00080109])

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)


In [76]:
def rmse(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

In [77]:
model = Sequential([
    Conv1D(filters=128, kernel_size=2, activation='relu', input_shape=(n_steps, X_train.shape[2])),
    MaxPooling1D(pool_size=2),
    Flatten(),
    Dense(50, activation='relu'),
    Dense(2)  # dos variables target
])

initial_learning_rate = 0.001
optimizer = Adam(learning_rate=initial_learning_rate)

huber_loss = Huber(delta=0.05)  # Puedes ajustar delta según sea necesario
model.compile(optimizer=optimizer, loss=huber_loss, metrics=[rmse])


In [78]:
early_stopping = EarlyStopping(
    monitor='rmse',  # Monitorea la pérdida de validación
    patience=15,         # Número de epochs sin mejora después del cual el entrenamiento será detenido
    restore_best_weights=True  # Restaura los pesos del modelo desde la epoch con la mejor pérdida de validación
)

history = model.fit(X_train, y_train, epochs=75, verbose=1, validation_split=0.15, callbacks=[early_stopping])


Epoch 1/75
125/125 [==============================] - 1s 5ms/step - loss: 0.0336 - rmse: 0.9840 - val_loss: 0.0337 - val_rmse: 0.9865
Epoch 2/75
125/125 [==============================] - 0s 4ms/step - loss: 0.0331 - rmse: 0.9648 - val_loss: 0.0337 - val_rmse: 0.9880
Epoch 3/75
125/125 [==============================] - 0s 4ms/step - loss: 0.0330 - rmse: 0.9713 - val_loss: 0.0337 - val_rmse: 0.9878
Epoch 4/75
125/125 [==============================] - 0s 3ms/step - loss: 0.0329 - rmse: 0.9738 - val_loss: 0.0338 - val_rmse: 0.9877
Epoch 5/75
125/125 [==============================] - 0s 3ms/step - loss: 0.0328 - rmse: 0.9649 - val_loss: 0.0339 - val_rmse: 0.9896
Epoch 6/75
125/125 [==============================] - 0s 3ms/step - loss: 0.0326 - rmse: 0.9625 - val_loss: 0.0339 - val_rmse: 0.9884
Epoch 7/75
125/125 [==============================] - 1s 4ms/step - loss: 0.0324 - rmse: 0.9630 - val_loss: 0.0341 - val_rmse: 0.9950
Epoch 8/75
125/125 [==============================] - 0s 4ms/s

In [79]:
y_test[]

1.022875503129459

In [82]:
y_test

array([[-0.52134184, -2.04956275],
       [ 0.37837541,  0.03100809],
       [ 0.6496745 ,  0.99063334],
       ...,
       [-0.124802  , -1.54911533],
       [ 1.49614491,  1.39923182],
       [ 0.58290435,  0.42737294]])

In [80]:
loss = model.evaluate(X_test, y_test)
print(f'Loss on test data: {loss}')


26/26 [==============================] - 0s 2ms/step - loss: 0.0405 - rmse: 1.1570
Loss on test data: [0.040507722645998, 1.1570314168930054]
